In [1]:
#| default_exp format
%load_ext autoreload
%autoreload 2

In [2]:
# | hide
import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))


In [3]:
# | export
import re
from typing import Iterable, Tuple, Union, List

import pandas as pd
import numpy as np
from fastcore.utils import listify
from fastcore.xtras import Path
from geopy.distance import geodesic
from rich.progress import Progress
from pyarrow import ArrowInvalid
from dotenv import load_dotenv, find_dotenv


from extracao.constants import BW, BW_pattern, APP_ANALISE_PT, APP_ANALISE_EN

RE_BW = re.compile(BW_pattern)
MAX_DIST = 10  # Km
LIMIT_FREQ = 84812.50
load_dotenv(find_dotenv())

True

In [4]:
# | export
def _read_df(folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Lê o dataframe formado por folder / stem.[parquet.gzip | fth | xslx]"""
    file = Path(f"{folder}/{stem}.parquet.gzip")
    try:
        df = pd.read_parquet(file)
    except (ArrowInvalid, FileNotFoundError) as e:
        raise e(f"Error when reading {file}") from e
    return df


# Formatação

> Este módulo possui funções auxiliares de formatação dos dados das várias fontes.


In [5]:
# | export
def parse_bw(
    bw: str,  # Designação de Emissão (Largura + Classe) codificada como string
) -> Tuple[str, str]:  # Largura e Classe de Emissão
    """Parse the bandwidth string"""
    if match := re.match(RE_BW, bw):
        multiplier = BW[match.group(2)]
        if mantissa := match.group(3):
            number = float(f"{match.group(1)}.{mantissa}")
        else:
            number = float(match.group(1))
        classe = match.group(4)
        return str(multiplier * number), str(classe)
    return "-1", "-1"


In [6]:
# | export
def _filter_matlab(
    df: pd.DataFrame,  # Arquivo de Dados Base de Entrada
) -> pd.DataFrame:  # Arquivo de Dados formatado para leitura no Matlab
    """Recebe a base de dados da Anatel e formata as colunas para leitura de acordo com os requisitos do Matlab"""
    df["#Estação"] = df["Número_Estação"]
    df.loc[df.Multiplicidade != "1", "#Estação"] = (
        df.loc[df.Multiplicidade != "1", "Número_Estação"]
        + "+"
        + df.loc[df.Multiplicidade != "1", "Multiplicidade"]
    )
    cols_desc = [
        "Fonte",
        "Status",
        "Classe",
        "Entidade",
        "Fistel",
        "#Estação",
        "Município_IBGE",
        "UF",
    ]
    df.loc[:, cols_desc].fillna("NI", inplace=True)

    df["Descrição"] = (
        "["
        + df.Fonte
        + "] "
        + df.Status
        + ", "
        + df.Classe
        + ", "
        + df.Entidade.str.title()
        + " ("
        + df.Fistel
        + ", "
        + df["#Estação"]
        + "), "
        + df.Município_IBGE
        + "/"
        + df.UF
    )

    bad_coords = df.Coords_Valida_IBGE == "0"

    df.loc[bad_coords, "Descrição"] = df.loc[bad_coords, "Descrição"] + "*"

    df.loc[bad_coords, ["Latitude", "Longitude"]] = df.loc[
        bad_coords, ["Latitude_IBGE", "Longitude_IBGE"]
    ].values

    df = df.loc[:, APP_ANALISE_PT]
    df.columns = APP_ANALISE_EN
    return df

In [7]:
# | export
def _format_matlab(
    df: pd.DataFrame,  # Arquivo de Dados Base de Entrada
) -> pd.DataFrame:  # Arquivo de Dados formatado para leitura no Matlab
    """Formata o arquivo final de dados para o formato esperado pela aplicação em Matlab"""
    df = df.astype("string")
    df.loc[len(df), :] = [
        "-1",
        "-15.7801",
        "-47.9292",
        "[TEMP] L, FX, Estação do SMP licenciada (cadastro temporário)",
        "10",
        "999999999",
        "NI",
        "-1",
    ]  # Paliativo...
    for c in ["Latitude", "Longitude"]:
        df[c] = df[c].fillna(-1).astype("float32")
    df["Frequency"] = df["Frequency"].astype("float64")
    df.loc[df.Service.isin(["", "-1"]), "Service"] = pd.NA
    df["Service"] = df.Service.fillna("-1").astype("int16")
    df.loc[df.Station.isin(["", "-1"]), "Station"] = pd.NA
    df["Station"] = df.Station.fillna("-1").astype("int32")
    df.loc[df.BW.isin(["", "-1"]), "BW"] = pd.NA
    df["BW"] = df["BW"].astype("float32").fillna(-1)
    df.loc[df["Class"].isin(["", "-1"]), "Class"] = pd.NA
    df["Class"] = df.Class.fillna("NI").astype("category")
    df = df[df.Frequency <= LIMIT_FREQ]
    df.sort_values(
        by=["Frequency", "Latitude", "Longitude", "Description"], inplace=True
    )
    unique_columns = df.columns.tolist().remove("Description")
    df = df.drop_duplicates(subset=unique_columns, keep="last").reset_index(drop=True)
    df["Id"] = [f"#{i+1}" for i in df.index]
    df["Id"] = df.Id.astype("string")
    df.loc[df.Description == "", "Description"] = pd.NA
    df["Description"] = df["Description"].astype("string").fillna("NI")
    return df[["Id"] + list(APP_ANALISE_EN)]

### Mesclagem
Função auxiliar para mesclar registros que são iguais das diversas bases, i.e. estão a uma distância menor que `MAX_DIST` e verificar a validade da mesclagem

In [8]:
# | export
def merge_close_rows(df_left, df_right):
    """Mescla os registros dos DataFrames `df_left` e `df_right` que estão a uma distância menor que MAX_DIST"""
    df1 = df_left.copy().reset_index(drop=True)
    df2 = df_right.copy().reset_index(drop=True)
    columns = ["Frequency", "Latitude", "Longitude"]
    for c in columns:
        df1[c] = df1[c].astype("float")
        df2[c] = df2[c].astype("float")
    df1.sort_values(columns, inplace=True)
    df2.sort_values(columns, inplace=True)
    with Progress(transient=True, refresh_per_second=2) as progress:
        task_left = progress.add_task(
            "[red]Iterando Tabela Principal...", total=len(df1)
        )
        for left in df1.itertuples():
            for right in df2[np.isclose(df2.Frequency, left.Frequency)].itertuples():
                if (
                    geodesic(
                        (left.Latitude, left.Longitude),
                        (right.Latitude, right.Longitude),
                    ).km
                    <= MAX_DIST
                ):
                    df1.loc[
                        left.Index, "Description"
                    ] = f"{left.Description} | {right.Description}"
                    df2 = df2.drop(right.Index)
                    break
            progress.update(
                task_left,
                advance=1,
                description=f"[green] Comparando Frequências {left.Frequency}MHz",
            )
        return pd.concat([df1, df2], ignore_index=True)


### Mesclagem (Vetorial)
Implementação vetorial da função anterior que mescla dois dataframes.

In [9]:
# | export
def get_boolean_cases(df, coords, suffixes) -> Tuple[pd.Series]:
    x, y = suffixes
    lat, long = coords

    right: pd.Series[bool] = (df[f"{lat}{x}"].isna() & df[f"{lat}{y}"].notna()) | (
        df[f"{long}{x}"].isna() & df[f"{long}{y}"].notna()
    )

    left: pd.Series[bool] = (df[f"{lat}{y}"].isna() & df[f"{lat}{x}"].notna()) | (
        df[f"{long}{y}"].isna() & df[f"{long}{x}"].notna()
    )

    both: pd.Series[bool] = np.all(
        df[[f"{lat}{x}", f"{long}{x}", f"{lat}{y}", f"{long}{y}"]].notna(),
        axis=1,
    )

    return left, right, both


def get_km_distance(row):
    return geodesic((row[0], row[1]), (row[2], row[3])).km


def merge_on_frequency(
    df_left: pd.DataFrame,  # DataFrame da esquerda a ser mesclado
    df_right: pd.DataFrame,  # DataFrame da direira a ser mesclado
    on: str = "Frequency",  # Coluna usada como chave de mesclagem
    coords: Tuple[str] = ("Latitude", "Longitude"),
    description: str = "Description",
    suffixes: Tuple[str] = ("_x", "_y"),  # Sufixo para as colunas que foram criadas
) -> pd.DataFrame:  # DataFrame resultante da mesclagem
    df: pd.DataFrame = pd.merge(
        df_left, df_right, on=on, how="outer", suffixes=suffixes, indicator=True
    )

    x, y = suffixes
    lat, long = coords

    left_cols: List[str] = [c for c in df.columns if y not in c]

    right_cols: List[str] = [c for c in df.columns if x not in c]

    left = df._merge == "left_only"
    right = df._merge == "right_only"
    both = df._merge == "both"

    only_left = df.loc[left, left_cols].drop_duplicates().reset_index(drop=True)
    only_left.columns = [c.removesuffix(x) for c in left_cols]

    only_right = df.loc[right, right_cols].drop_duplicates().reset_index(drop=True)
    only_right.columns = [c.removesuffix(y) for c in right_cols]

    both_columns = [f"{lat}{x}", f"{long}{x}", f"{lat}{y}", f"{long}{y}"]

    df.loc[both, "Distance"] = df.loc[both, both_columns].apply(get_km_distance, axis=1)

    close = df.loc[both, "Distance"] <= MAX_DIST
    df_close = df.loc[(both & close)].drop_duplicates().reset_index(drop=True)
    df_close[f"Description{x}"] = (
        df_close[f"Description{x}"] + " | " + df_close[f"Description{y}"]
    )
    df_close = df_close[left_cols]
    df_close.columns = only_left.columns

    df_far_left = (
        df.loc[(both & ~close), left_cols].drop_duplicates().reset_index(drop=True)
    )
    df_far_left.columns = only_left.columns

    df_far_right = (
        df.loc[(both & ~close), right_cols].drop_duplicates().reset_index(drop=True)
    )
    df_far_right.columns = only_right.columns

    merged_df = pd.concat(
        [df_left, df_right, df_close, df_far_right, df_far_left], ignore_index=True
    )
    merged_df.drop(columns=["_merge"], inplace=True)
    return _format_matlab(merged_df)

In [10]:
folder = Path.cwd().parent / "dados"

In [11]:
from extracao.reading import *


In [12]:
df_left = _filter_matlab(read_base(folder))
df_right = read_aero(folder)


In [13]:
df = merge_on_frequency(df_left, df_right)


In [14]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874720 entries, 0 to 874719
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   Id           874720 non-null  string  
 1   Frequency    874720 non-null  float64 
 2   Latitude     874720 non-null  float32 
 3   Longitude    874720 non-null  float32 
 4   Description  874720 non-null  string  
 5   Service      874720 non-null  int16   
 6   Station      874720 non-null  int32   
 7   Class        874720 non-null  category
 8   BW           874720 non-null  float32 
dtypes: category(1), float32(3), float64(1), int16(1), int32(1), string(2)
memory usage: 35.9 MB


In [15]:
df.sample(10)


,Id,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
524751,#524752,317.00000,-22.548611,-44.759998,"[MOS] L, FX, Telefonica Brasil S.A. (504171794...",19,1424955,F8E,1360.0
714359,#714360,1870.00000,-21.760834,-43.344444,"[MOS] L, FX, Telefonica Brasil S.A. (504171794...",19,697802159,G7W,200.0
34981,#34982,94.90000,-16.360001,-49.500198,"[MOS] FM-C2, E3, Cultura Fm Stereo Som Ltda (1...",230,323035310,NI,256.0
161998,#161999,156.67500,-22.368700,-41.775101,"[STEL] L, ML, Petroleo Brasileiro S A Petrobra...",604,1012639239,F3E,11.0
399881,#399882,168.38125,-19.562222,-44.066666,"[MOS] L, ML, Policia Militar Do Estado De Mina...",19,1008046741,F1W,11.0
410847,#410848,168.49375,-19.770111,-43.932564,"[MOS] L, FX, Policia Militar Do Estado De Mina...",19,1002203608,G1W,8.1
253392,#253393,165.35000,-9.805000,-37.683334,"[MOS] L, FX, Oi S.A. - Em Recuperacao Judicial...",175,487260,F3E,16.0
32494,#32495,87.90000,-13.659722,-57.890278,"[SRD] RADCOM, 3-B, Associacao Comunitaria Camp...",231,682991627,NI,256.0
248588,#248589,164.79000,-11.419306,-58.757751,"[MOS] L, FX, Oi S.A. - Em Recuperacao Judicial...",175,441540333,F3E,16.0
817796,#817797,8088.67000,-13.688722,-46.565834,"[MOS] L, FX, Oi S.A.- Em Recuperação Judicial ...",175,422353787,D7W,29700.0


In [16]:
df.head()


,Id,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
0,#1,-1.000,-15.780100,-47.929199,"[TEMP] L, FX, Estação do SMP licenciada (cadas...",10,999999999,NI,-1.0
1,#2,0.028,-22.662779,-43.476391,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",19,1557670,J9E,8.0
2,#3,0.030,-23.709999,-46.273335,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",19,1558412,J3E,2.0
3,#4,0.030,-23.441668,-46.590832,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",19,1557823,J3E,1.0
4,#5,0.030,-22.926666,-43.264999,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",19,859761,J3E,0.5


In [49]:
df_left.shape


(818001, 8)

## Otimização dos Tipos de dados
A serem criados dataframes, normalmente a tipo de data é aquele com maior resolução possível, nem sempre isso é necessário, os arquivos de espectro mesmo possuem somente uma casa decimal, portanto um `float16` já é suficiente para armazená-los. As funções a seguir fazem essa otimização

Code below borrowed from https://medium.com/bigdatarepublic/advanced-pandas-optimize-speed-and-memory-a654b53be6c2

In [ ]:
# | export
def optimize_floats(
    df: pd.DataFrame,  # DataFrame a ser otimizado
    exclude: Iterable[str] = None,  # Colunas a serem excluidas da otimização
) -> pd.DataFrame:  # DataFrame com as colunas do tipo `float` otimizadas
    """Otimiza os floats do dataframe para reduzir o uso de memória"""
    floats = df.select_dtypes(include=["float64"]).columns.tolist()
    floats = [c for c in floats if c not in listify(exclude)]
    df[floats] = df[floats].apply(pd.to_numeric, downcast="float")
    return df


In [ ]:
# | export
def optimize_ints(
    df: pd.DataFrame,  # Dataframe a ser otimizado
    exclude: Iterable[str] = None,  # Colunas a serem excluidas da otimização
) -> pd.DataFrame:  # DataFrame com as colunas do tipo `int` otimizadas
    """Otimiza os ints do dataframe para reduzir o uso de memória"""
    ints = df.select_dtypes(include=["int64"]).columns.tolist()
    ints = [c for c in ints if c not in listify(exclude)]
    df[ints] = df[ints].apply(pd.to_numeric, downcast="integer")
    return df


In [ ]:
# | export
def optimize_objects(
    df: pd.DataFrame,  # DataFrame a ser otimizado
    datetime_features: Iterable[
        str
    ] = None,  # Colunas que serão convertidas para datetime
    exclude: Iterable[str] = None,  # Colunas que não serão convertidas
) -> pd.DataFrame:  # DataFrame com as colunas do tipo `object` otimizadas
    """Otimiza as colunas do tipo `object` no DataFrame para `category` ou `string` para reduzir a memória e tamanho de arquivo"""
    exclude = listify(exclude)
    datetime_features = listify(datetime_features)
    for col in df.select_dtypes(
        include=["object", "string", "category"]
    ).columns.tolist():
        if col not in datetime_features:
            if col in exclude:
                continue
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if float(num_unique_values) / num_total_values < 0.5:
                dtype = "category"
            else:
                dtype = "string"
            df[col] = df[col].astype(dtype)
        else:
            df[col] = pd.to_datetime(df[col]).dt.date
    return df


In [ ]:
# | export
def df_optimize(
    df: pd.DataFrame,  # DataFrame a ser otimizado
    datetime_features: Iterable[
        str
    ] = None,  # Colunas que serão convertidas para datetime
    exclude: Iterable[str] = None,  # Colunas que não serão convertidas
) -> pd.DataFrame:  # DataFrame com as colunas com tipos de dados otimizados
    """Função que encapsula as anteriores para otimizar os tipos de dados e reduzir o tamanho do arquivo e uso de memória"""
    if datetime_features is None:
        datetime_features = []
    return optimize_floats(
        optimize_ints(optimize_objects(df, datetime_features, exclude), exclude),
        exclude,
    )
